In [1]:
from transformers import pipeline
import pandas as pd
from tqdm import tqdm

/home/tarsojabbes/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Criação do Pipeline com o Modelo

In [2]:
PATH = "../model"
classifier = pipeline('text-classification', model= PATH, tokenizer= PATH)

In [3]:
df_all = pd.read_csv('../Dados/test_data.csv')

In [4]:
df_all.head()

,query,label
0,"A gente xinga a grande mídia o tempo todo, e...",0
1,g1: União Europeia ameaça com sanções países q...,0
2,é engraçado e retrata um pouco do lado 'marg...,0
3,o lula nunca acertou a nota azul do guitar hero,1
4,"deve ser foda namorar uma evangélica, vc manda...",1


In [5]:
classifier(df_all['query'][0])

[{'label': 'LABEL_0', 'score': 0.9974229335784912}]

## Rodando o classificador

In [7]:
preds = []

for i, query in enumerate(tqdm(df_all['query'])):
    try:
        pred = classifier(query)
        if pred[0]['label'] == 'LABEL_1' and pred[0]['score'] > 0.8:
            preds.append(df_all['query'][i])    
    except RuntimeError as e:
        print(f"query: {query}")

100%|██████████| 1702/1702 [02:18<00:00, 12.25it/s]


In [8]:
len(preds)

514

In [9]:
df_all['classification'] = df_all['query'].apply(lambda x: 1 if x in preds else 0)

In [10]:
df_all.to_csv('../Dados/results_data2.csv', index=False)

## Classificação para exemplos extras criados

In [12]:
df_custom = pd.read_csv('../Dados/custom.csv')

preds = []

for i, query in enumerate(tqdm(df_custom['query'])):
    try:
        pred = classifier(query)
        if pred[0]['label'] == 'LABEL_1' and pred[0]['score'] > 0.8:
            preds.append(df_custom['query'][i])    
    except RuntimeError as e:
        print(f"query: {query}")

df_custom['classification'] = df_custom['query'].apply(lambda x: 1 if x in preds else 0)
df_custom.to_csv('../Dados/custom_result.csv', index=False)


100%|██████████| 20/20 [00:01<00:00, 18.39it/s]


In [14]:
df_custom_result = pd.read_csv("../Dados/custom_result.csv")

df_custom_result['acerto'] = df_custom_result['label'] == df_custom_result['classification']

acertos = df_custom_result['acerto'].sum()

erros = len(df_custom_result) - acertos

print(f"Acertos: {acertos}")
print(f"Erros: {erros}")

porcentagem_acertos = acertos / len(df_custom_result) * 100
print(f"Porcentagem de Acertos: {porcentagem_acertos:.2f}%")

Acertos: 19
Erros: 1
Porcentagem de Acertos: 95.00%
